<a href="https://colab.research.google.com/github/saipragna25/Special_Topics_LLM_finetuning_assignment/blob/main/task%20G/ST_LLM_Finetuning_g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains Quantization of llm with ggml and gguf

Quantizing llm with ggml

In [1]:
!nvidia-smi

Mon Nov 20 18:12:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [3]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2338898 sha256=2ad174de4e2a45e2c503089cf0a9c694eb18d2b86b05f7dac458984b86f8a2a0
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers


Let's Choose the model

In [1]:
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [2]:
from ctransformers import AutoModelForCausalLM

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(model_id,
                                           model_type="llama",
                                           #lib='avx2', for cpu use
                                           gpu_layers=130, #110 for 7b, 130 for 13b
                                           **config
                                           )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

LLAMA-2-7B-Chat tests

Tokenizer

In [3]:
prompt="""Write a short story that introduces the first 10 Presidents of the United States, giving each President a dedicated sentence.
The story should weave together historical facts and engaging narrative elements to make each President memorable."""

In [4]:
tokens = llm.tokenize(prompt)

In [5]:
tokens

[1,
 6113,
 263,
 3273,
 5828,
 393,
 4547,
 778,
 278,
 937,
 29871,
 29896,
 29900,
 4360,
 16719,
 310,
 278,
 3303,
 3900,
 29892,
 6820,
 1269,
 7178,
 263,
 16955,
 10541,
 29889,
 29871,
 13,
 1576,
 5828,
 881,
 591,
 1351,
 4208,
 15839,
 17099,
 322,
 3033,
 6751,
 15474,
 1230,
 3161,
 304,
 1207,
 1269,
 7178,
 26959,
 519,
 29889]

pipeline execution

In [6]:
llm(prompt, stream=False)

"\n\nHere is the list of the first 10 Presidents:\n\n1. George Washington\n2. John Adams\n3. Thomas Jefferson\n4. James Madison\n5. James Monroe\n6. John Quincy Adams\n7. Andrew Jackson\n8. Martin Van Buren\n9. William Henry Harrison\n10. John Tyler\n\nThe story should be set in the early 19th century, and each President's sentence should reflect their personality, accomplishments, or notable events that occurred during their term.\n\nHere is a sample of how you might begin the story:\n\nIt was a brisk winter morning in 1801 when George Washington, the first President of the United States, stepped down from the presidency and handed the reins to his successor, John Adams. As the new administration took office, tensions between the young nation and Europe were high, and Adams found himself facing a daunting challenge: keeping America out of another costly war.\n\nAs you write your story, consider including details such as these:\n\n* George Washington's decision to set a precedent by re

In [7]:
prompt2 = """¿Cuándo y por quién fue inventada la penicilina?"""
llm(prompt2, stream=False)

'\n\nLa penicilina es un antibiótico que se utiliza para tratar infecciones bacterianas. Fue descubierta en 1928 por el científico británico Alexander Fleming.\n\nFleming trabajaba en su laboratorio en el Hospital de St. Mary en Londres, cuando observó que una colonia de hifas de un cultivo de bacterias (Streptococcus) había sido afectada por una sustancia desconocida. Después de investigar más a fondo, descubrió que la sustancia era una cierta forma de moldo (Penicillium notatum), que había crecido en su laboratorio y había matado las bacterias circundantes.\n\nFleming comenzó a investigar la sustancia desconocida y descubrió que tenía un poder antibacteriano. Luego, began to investigate the substance and discovered that it had powerful antibacterial properties.\n\nEn 1941, el médico australiano Howard Florey y su equipo de investigación pusieron en marcha las primeras pruebas de la pen'

Now, Let's generate with stream execution

In [11]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 11.5 

In [8]:
import gradio as gr
from ctransformers import AutoModelForCausalLM
import time


model_id = model_id
llm = AutoModelForCausalLM.from_pretrained(model_id, model_type="llama", gpu_layers=130)

def generate_response(prompt):
    start = time.time()
    NUM_TOKENS = 0
    response = ""
    tokens = llm.tokenize(prompt)

    for token in llm.generate(tokens):
        response += llm.detokenize(token)
        NUM_TOKENS += 1
    time_generate = time.time() - start
    return response, NUM_TOKENS, time_generate

iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs=["text", "number", "number"]
)

iface.launch(debug = True)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://389631399f45b29e8a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://389631399f45b29e8a.gradio.live
